In [ ]:
from gdpyt import GdpytImageCollection, GdpytCalibrationSet
from os.path import join
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage.morphology import disk, square
%matplotlib notebook

In [ ]:
meas_path = r'C:\Users\silus\Box Sync\master_thesis\results\deflection\8_4_2020\left_electrode\0005\loc4\IC10'
# Measurement at 1000 V
v = 1000
test_img_path = join(meas_path, '{}V'.format(v), 'run1', 'images')
calib_img_path = join(meas_path, 'calibration_images')

In [ ]:
filetype = '.tif'
# None: Take any shape. 
# Number > 0: Only keep particles for which 4*pi*area / circumference**2 - 1 is smaller than the value given
shape_tolerance = 0.4 

# Minimum and maximum particle size (areas) in pixel
min_p_size = 5
max_p_size = 3000

# Tolerance in pixels for the amount that xy position of the center of a particle can vary over the images and
# still get the the same id
same_id_thresh = 5

# Add filters if needed
processing = {
    #'median': {'args': [disk(3)]},
    #'gaussian': {'args': [], 'kwargs': dict(sigma=1, preserve_range=True)}
}

# Multiple thresholding algorithms are possible, ostu usually works the best
threshold = {
    #'multiotsu': {'classes': 3}
    #'triangle': []
    'otsu': []
    #'local': {'block_size': 51, 'offset': -10, 'method': 'mean'} 
}

calib_col = GdpytImageCollection(calib_img_path, filetype, processing_specs=processing, thresholding_specs=threshold,
                                 min_particle_size=min_p_size, max_particle_size=max_p_size, shape_tol=shape_tolerance)
calib_col.uniformize_particle_ids(threshold=same_id_thresh)

# Calibration image filename to z position dictionary
# The example here maps 'calib_X.tif' to z = X
name_to_z = {}
for image in calib_col.images.values():
    name_to_z.update({image.filename: float(image.filename.split('_')[-1].split('.')[0])})

# Dilate: dilate images to have room to slide template and find ideal position
calib_set = calib_col.create_calibration(name_to_z, dilate=False) 

In [ ]:
fig = calib_col.plot(raw=True)
fig.subplots_adjust(hspace=0.1)
fig.show()

In [ ]:
# Plot single images
# Replace with valid filename
fig, axes = plt.subplots(ncols=3, figsize=(9,4))
axes[0].imshow(calib_col.images['calib_50.tif'].raw)
axes[0].set_title('Raw image')
axes[1].imshow(calib_col.images['calib_50.tif'].filtered)
axes[1].set_title('Filtered image')
axes[2].imshow(calib_col.images['calib_50.tif'].draw_particles(raw=True))
axes[2].set_title('Raw image with particles')


## Load test images

In [ ]:
test_col = GdpytImageCollection(test_img_path, filetype, processing_specs=processing, thresholding_specs=threshold,
                             min_particle_size=min_p_size, max_particle_size=max_p_size, shape_tol=shape_tolerance)
test_col.uniformize_particle_ids(baseline=calib_set, threshold=same_id_thresh)

In [ ]:
# Show test collection (but might be a lot of images)

#fig = test_col.plot(raw=True)
#fig.subplots_adjust(hspace=0.1)
#fig.show()

In [ ]:
# Infer position using Barnkob's similarity function ('bccorr')
# Other options: 'znccorr' (zero-normalized cross-corr.) or 'ncorr' (normalized cross-corr.)
test_col.infer_z(calib_set).bccorr(min_cm=0.95)

## Visualize deflection

In [ ]:
plt.figure()
plt.imshow(test_col.images['0005_left_loc4_500V_1_X102.tif'].draw_particles())

In [ ]:
sort_imgs = lambda x: int(x.split('_X')[-1].split('.')[0])
# Pass ids that should be displayed
fig = test_col.plot_particle_coordinate(particle_ids=[i for i in range(50)], sort_images=sort_imgs)
fig.show()

## Getting coordinates of each particle in each frame as one dataframe

In [ ]:
sort_images = sort_imgs

coordinates= []
if sort_images is None:
    for image in test_cols[v].images.values():
        coordinates.append(image.particle_coordinates())
else:
    if not callable(sort_images):
        raise TypeError("sort_images must be a function that takes an image name as an argument and returns a value"
                        "that can be used to sort the images")
    # Get the particle coordinates from all the images
    for file in sorted(test_cols[v].files, key=sort_images):
        coordinates.append(test_cols[v].images[file].particle_coordinates().set_index('id'))
coordinates = pd.concat(coordinates, keys=[i for i in range(len(coordinates))], names=['Frame'])

coordinates.head()